<table style="border: none" align="left">
   <tr style="border: none">
      <th style="border: none"><font face="verdana" size="5" color="black"><b>Twitter sentiment analysis with IBM Watson Machine Learning</b></th>
      <th style="border: none"><img src="https://github.com/pmservice/customer-satisfaction-prediction/blob/master/app/static/images/ml_icon_gray.png?raw=true" alt="Watson Machine Learning icon" height="40" width="40"></th>
   <tr style="border: none">
       <th style="border: none"><img src="https://github.com/pmservice/wml-sample-models/blob/master/spark/sentiment-prediction/images/sentiment_banner_3-04.png?raw=true" width="600" alt="Icon"> </th>
   </tr>
</table>

This notebook contains steps and code to develop a predictive model, and start scoring new data. This notebook introduces commands for getting data and for basic data cleaning and exploration, pipeline creation, model training, model persistance to Watson Machine Learning repository, model deployment, and scoring.

Some familiarity with Python is helpful. This notebook uses Python 2.0 and Apache® Spark 2.0.

You will use a data set named **Twitter Sentiment**, which includes many tweets and their binary labels (e.g. POSITIVE or NEGATIVE). Use the details of this data set to predict sentiment of tweets.

## Learning goals

The learning goals of this notebook are:

-  Load a CSV file into an Apache® Spark DataFrame.
-  Explore data.
-  Prepare data for training and evaluation.
-  Create an Apache® Spark machine learning pipeline.
-  Train and evaluate a model.
-  Persist a pipeline and model in Watson Machine Learning repository.
-  Explore and visualize prediction result using the plotly package.
-  Deploy a model for streaming using Wastson Machine Learning API.
-  Working with Bluemix MessageHub.


## Contents

This notebook contains the following parts:

1.	[Setup](#setup)
2.	[Load and explore data](#load)
3.	[Create spark ml model](#model)
4.	[Persist model](#persistence)
5.	[Predict locally and visualize](#visualization)
6.	[Deploy and score in a Cloud](#scoring)
7.	[Summary and next steps](#summary)

<a id="setup"></a>
## 1. Setup

Before you use the sample code in this notebook, you must perform the following setup tasks:

### Required Bluemix services

-  Create a [Watson Machine Learning Service](https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/) instance (a free plan is offered). 
-  Create a [MessageHub Service](https://console.bluemix.net/catalog) instance (free plan is offered).
-  Create two topics in MessageHub: **streamingi** and **streamingo**. 
-  Make sure that you are using a Spark 2.0 kernel.

**Note**:  Get your credentials from Manage Credentials section of Watson Machine Learning and MessageHub Service.

### Twitter Sentiment Data

The Twitter Sentiment data is available on [project's Github page](https://github.com/pmservice/wml-sample-models/tree/master/spark/sentiment-prediction).

<a id="load"></a>
## 2. Load and explore data

In this section you will load the data as an Apache® Spark DataFrame and perform a basic exploration.

**Tip**: First, you need to install required packages. You can do it by running the following code. You can run the code below after changing **cell type** to `code`.

In [1]:
import wget
from pprint import PrettyPrinter

pp = PrettyPrinter(indent=2, depth=3).pprint

In [2]:
link_to_data = 'https://github.com/pmservice/wml-sample-models/raw/master/spark/sentiment-prediction/data/trainingTweets.csv'
filename = wget.download(link_to_data)

print(filename)

trainingTweets.csv


The csv file trainingTweets.csv is availble on gpfs now. Load the file to Apache® Spark DataFrame using code below.

In [3]:
spark = SparkSession.builder.getOrCreate()

df_data = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .option('inferSchema', 'true')\
  .load(filename)

Explore the loaded data by using Apache® Spark DataFrame methods:
-  print schema
-  print first ten records
-  count all records

In [4]:
df_data.printSchema()

root
 |-- id: integer (nullable = true)
 |-- text: string (nullable = true)
 |-- label: integer (nullable = true)



As you can see, the data contains 3 fields. ``label`` field is the one we would like to classify tweets.

In [5]:
df_data.show(n=10)

+---+--------------------+-----+
| id|                text|label|
+---+--------------------+-----+
|  1|realdonaldtrump s...|    1|
|  2|cnnvideo hillaryc...|    1|
|  3|realdonaldtrump t...|    1|
|  4|sensanders the am...|    1|
|  5|billburton realdo...|    0|
|  6|reince hillarycli...|    0|
|  7|bentechpro realdo...|    1|
|  8|dahbigj hopeobama...|    0|
|  9|theosmelek thuddy...|    0|
| 10|realdonaldtrump r...|    0|
| 11|jnjinnovation nee...|    0|
| 12|sen tedcruz halbi...|    1|
| 13|progressiveswin t...|    1|
| 14|mclem retward rea...|    1|
| 15|sometimes the onl...|    0|
| 16|cassandra17lina e...|    1|
| 17|realdonaldtrump d...|    1|
| 18|willmcavoyacn hey...|    0|
| 19|hillaryclinton i ...|    0|
| 20|realdonaldtrump d...|    0|
+---+--------------------+-----+
only showing top 20 rows



In [6]:
print("Total number of records: {count}".format(count=df_data.count()))

Total number of records: 5987


Data set contains 5987 records.

<a id="model"></a>
## 3. Create an Apache® Spark machine learning model

In this section you will learn how to prepare data, create an Apache® Spark machine learning pipeline, and train a model.

### 3.1: Prepare data

In this subsection you will split your data into: 
-  The train data set, which is the largest group, is used for training.
-  The test data set will be used for model evaluation and is used to test the assumptions of the model.
-  The predict data set will be used for prediction.

In [7]:
splitted_data = df_data.randomSplit([0.8, 0.18, 0.02], 24)
train_data = splitted_data[0]
test_data = splitted_data[1]
predict_data = splitted_data[2]

print("Number of training records: {count}".format(count=train_data.count()))
print("Number of testing records: {count}".format(count=test_data.count()))
print("Number of prediction records: {count}".format(count=predict_data.count()))

Number of training records: 4783
Number of testing records: 1076
Number of prediction records: 128


### 3.2: Create pipeline and train a model

In this section you will create an Apache® Spark machine learning pipeline and then train the model.

In the first step you need to import the Apache® Spark machine learning packages that will be needed in the subsequent steps.

In [8]:
from pyspark.ml.feature import Tokenizer, OneHotEncoder, StringIndexer, HashingTF
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml import Pipeline, Model

In the data preprocessing step, convert all the string fields to numeric ones by using **Tokenizer** and then **HashingTF** transformer.

In [9]:
tokenizer = Tokenizer(inputCol="text", outputCol="words")
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")

Next, define estimator you want to use for classification. Logistic Regression is used in the following example.

In [10]:
lr = LogisticRegression(maxIter=10, regParam=0.01)

Let's build the pipeline now. A pipeline consists of transformers and an estimator.

In [11]:
pipeline_lr = Pipeline(stages=[tokenizer, hashingTF, lr])

Now, you can train your Logistic Regression model by using previously defined **pipeline** and **train data**.

In [12]:
model_lr = pipeline_lr.fit(train_data)

You can evaluate the model on the test data. Area under ROC will be used as evaluation metrics.

In [13]:
predictions = model_lr.transform(test_data)
evaluatorRF = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="prediction", metricName="areaUnderROC")
accuracy = evaluatorRF.evaluate(predictions)

print("Accuracy = {acc:4.3f}".format(acc=accuracy))

Accuracy = 0.762


Now you can tune your model to achieve better accuracy. For simplicity of this notebook, tuning section is omitted.

<a id="persistence"></a>
## 4. Persist model

In this section you will learn how to store your pipeline and model in Watson Machine Learning repository by using python client libraries.

First, you must import Watson Machine Learning client libraries.

**Note**: Apache® Spark 2.0 or higher is required.

In [14]:
from repository.mlrepositoryclient import MLRepositoryClient
from repository.mlrepositoryartifact import MLRepositoryArtifact
from repository.mlrepository import MetaProps

Authenticate to Watson Machine Learning service on Bluemix.

**Action**: Put credentials from your instance of **Watson Machine Learning** service in cell bellow.

In [15]:
wml_credentials={
  "url": "https://ibm-watson-ml.mybluemix.net",
  "access_key": "****",
  "username": "****",
  "password": "****",
  "instance_id": "****"
}

Create WML client and authorize it.

In [16]:
ml_repository_client = MLRepositoryClient(wml_credentials['url'])
ml_repository_client.authorize(wml_credentials['username'], wml_credentials['password'])

Create model artifact (abstraction layer).

In [17]:
model_artifact = MLRepositoryArtifact(model_lr, training_data=train_data, name="Sentiment Prediction Model" )

**Tip**: The MLRepositoryArtifact method expects a trained model object, training data, and a model name. (It is this model name that is displayed by the Watson Machine Learning service).

### 4.1: Save pipeline and model

In this subsection you will learn how to save pipeline and model artifacts to your Watson Machine Learning instance.

In [18]:
saved_model = ml_repository_client.models.save(model_artifact)

Get saved model metadata from Watson Machine Learning.

**Tip**: Use *meta.available_props()* to get the list of available props.

In [19]:
pp(saved_model.meta.available_props())

[ 'inputDataSchema',
  'evaluationMetrics',
  'pipelineVersionHref',
  'modelVersionHref',
  'trainingDataRef',
  'pipelineType',
  'creationTime',
  'lastUpdated',
  'label',
  'authorEmail',
  'trainingDataSchema',
  'authorName',
  'version',
  'modelType',
  'runtime',
  'evaluationMethod']


In [20]:
print("modelType: {model_type}".format(model_type=saved_model.meta.prop("modelType")))
print("trainingDataSchema: {data_schema}".format(data_schema=saved_model.meta.prop("trainingDataSchema")))
print("creationTime: {ctime}".format(ctime=saved_model.meta.prop("creationTime")))
print("modelVersionHref: {href}".format(href=saved_model.meta.prop("modelVersionHref")))
print("label: {label}".format(label=saved_model.meta.prop("label")))

modelType: sparkml-model-2.0
trainingDataSchema: {u'fields': [{u'nullable': True, u'type': u'integer', u'name': u'id', u'metadata': {}}, {u'nullable': True, u'type': u'string', u'name': u'text', u'metadata': {}}, {u'nullable': True, u'type': u'integer', u'name': u'label', u'metadata': {}}], u'type': u'struct'}
creationTime: 2017-09-15 12:09:26.167000+00:00
modelVersionHref: https://ibm-watson-ml.mybluemix.net/v2/artifacts/models/c575aac9-1dad-4e29-98d4-1f8c7dc8c764/versions/58d2fe5b-7661-459b-b342-a262f5846e61
label: label


**Tip**: **modelVersionHref** is our model unique indentifier in the Watson Machine Learning repository.

### 4.2: Load model

In this subsection you will learn how to load back saved model from specified instance of Watson Machine Learning.

In [21]:
loadedModelArtifact = ml_repository_client.models.get(saved_model.uid)

You can print for example model name to make sure that model has been loaded correctly.

In [22]:
print(loadedModelArtifact.name)

Sentiment Prediction Model


As you can see the name is correct. You have already learned how save and load the model from Watson Machine Learning repository.

<a id="visualization"></a>
## 5. Predict locally and visualize

In this section you will learn how to score test data using loaded model and visualize the prediction results with plotly package.

### 5.1: Make local prediction using previously loaded model and test data

In this subsection you will score ``predict_data`` data set.

In [23]:
predictions = loadedModelArtifact.model_instance().transform(predict_data)

Preview the results by calling the *show()* method on the predictions DataFrame.

In [24]:
predictions.show(5)

+---+--------------------+-----+--------------------+--------------------+--------------------+--------------------+----------+
| id|                text|label|               words|            features|       rawPrediction|         probability|prediction|
+---+--------------------+-----+--------------------+--------------------+--------------------+--------------------+----------+
|254|realdonaldtrump t...|    1|[realdonaldtrump,...|(262144,[4312,961...|[-1.3150612511415...|[0.21164114610918...|       1.0|
|256|realdonaldtrump j...|    0|[realdonaldtrump,...|(262144,[14,13396...|[5.48207446382385...|[0.99585655006955...|       0.0|
|296|realdonaldtrump t...|    1|[realdonaldtrump,...|(262144,[15889,21...|[-1.3175186267989...|[0.21123142555360...|       1.0|
|312|sensanders keep t...|    0|[sensanders, keep...|(262144,[32890,91...|[3.29411163264101...|[0.96422625174639...|       0.0|
|362|katiedaviscourt i...|    0|[katiedaviscourt,...|(262144,[16332,21...|[3.83815328742005...|[0.978920

By tabulating a count, you can see the split by sentiment.

In [25]:
predictions.select("label").groupBy("label").count().show()

+-----+-----+
|label|count|
+-----+-----+
|    1|   49|
|    0|   79|
+-----+-----+



### 5.2: Sample visualization of data with Plotly package

In this subsection you will explore prediction results with Plotly, which is an online analytics and data visualization tool.

**Tip**: First, you need to install required packages. You can do it by running the following code. Run it only one time.

In [26]:
import sys
import pandas
import plotly.plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf
import plotly.graph_objs as go

init_notebook_mode(connected=True)
sys.path.append("".join([os.environ["HOME"]])) 

You have to convert the Apache Spark DataFrame to a Pandas DataFrame to be used by ploting function.

In [27]:
predictions_pdf = predictions.select("prediction", "label", "text").toPandas()
cumulative_stats = predictions_pdf.groupby(['label']).count()
labels_data_plot = cumulative_stats.index
values_data_plot = cumulative_stats['text']

Plot a pie chart that shows the predicted sentiment label.

In [28]:
product_data = [go.Pie(
            labels=labels_data_plot,
            values=values_data_plot,
    )]

product_layout = go.Layout(
    title='Sentiment',
)

fig = go.Figure(data=product_data, layout=product_layout)
iplot(fig)

<a id="scoring"></a>
## 6. Deploy and score in a Cloud

In this section you will learn how to create streaming deployment by using the Watson Machine Learning REST API and work with Bluemix MessageHub. 
For more information about REST APIs, see the [Swagger Documentation](http://watson-ml-api.mybluemix.net/).

To work with the Watson Machine Leraning REST API you must generate an access token. To do that you can use the following sample code:

In [29]:
import urllib3, requests, json, base64

headers = urllib3.util.make_headers(basic_auth='{username}:{password}'.format(username=wml_credentials['username'], password=wml_credentials['password']))
url = '{url}/v3/identity/token'.format(url=wml_credentials['url'])
response = requests.get(url, headers=headers)
mltoken = json.loads(response.text).get('token')

### 6.1: Create streaming deployment

#### Get published models url from instance details

In [30]:
endpoint_instance = "{url}/v3/wml_instances/{instance_id}".format(url=wml_credentials['url'], instance_id=wml_credentials['instance_id'])
header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + mltoken}

response_get_instance = requests.get(endpoint_instance, headers=header)

print(response_get_instance)
pp(json.loads(response_get_instance.text))

<Response [200]>
{ u'entity': { u'account': { u'id': u'8dba8395f5b8dc6fcab49d43161ab9f1',
                             u'name': u"Wojciech Sobala's Account",
                             u'type': u'TRIAL'},
               u'deployments': { u'url': u'https://ibm-watson-ml.mybluemix.net/v3/wml_instances/a153b37a-2b65-47e6-b0ef-2d71da079c4f/deployments'},
               u'organization_guid': u'a7b3ef43-5dc2-41fc-8fa7-29b0846e5807',
               u'owner': { u'beta_user': True,
                           u'country_code': u'POL',
                           u'email': u'wojciech.sobala@pl.ibm.com',
                           u'ibm_id': u'3100020VJR',
                           u'user_id': u'1c54455e-c23b-4506-b184-27494b507536'},
               u'plan': u'free',
               u'plan_id': u'3f6acf43-ede8-413a-ac69-f8af3bb0cbfe',
               u'published_models': { u'url': u'https://ibm-watson-ml.mybluemix.net/v3/wml_instances/a153b37a-2b65-47e6-b0ef-2d71da079c4f/published_models'},
       

In [31]:
endpoint_published_models = json.loads(response_get_instance.text).get('entity').get('published_models').get('url')

print(endpoint_published_models)

https://ibm-watson-ml.mybluemix.net/v3/wml_instances/a153b37a-2b65-47e6-b0ef-2d71da079c4f/published_models


Execute the following sample code that uses the published_models endpoint to get deployments url.

#### Get the list of published models

In [32]:
header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + mltoken}
response_get = requests.get(endpoint_published_models, headers=header)

print(response_get)
pp(json.loads(response_get.text).get('resources'))

<Response [200]>
[ { u'entity': { u'author': {},
                 u'deployments': {...},
                 u'input_data_schema': {...},
                 u'label_col': u'label',
                 u'latest_version': {...},
                 u'model_type': u'sparkml-model-2.0',
                 u'name': u'Sentiment Prediction Model',
                 u'runtime_environment': u'spark-2.0',
                 u'training_data_schema': {...}},
    u'metadata': { u'created_at': u'2017-09-15T12:09:26.167Z',
                   u'guid': u'c575aac9-1dad-4e29-98d4-1f8c7dc8c764',
                   u'modified_at': u'2017-09-15T12:09:26.276Z',
                   u'url': u'https://ibm-watson-ml.mybluemix.net/v3/wml_instances/a153b37a-2b65-47e6-b0ef-2d71da079c4f/published_models/c575aac9-1dad-4e29-98d4-1f8c7dc8c764'}}]


In [33]:
for x in json.loads(response_get.text).get('resources'):
    if (x.get('metadata').get('guid') == saved_model.uid):
        endpoint_deployments = x.get('entity').get('deployments').get('url') 

print(endpoint_deployments)

https://ibm-watson-ml.mybluemix.net/v3/wml_instances/a153b37a-2b65-47e6-b0ef-2d71da079c4f/published_models/c575aac9-1dad-4e29-98d4-1f8c7dc8c764/deployments


#### Create streaming deployment for published model

**Action**: Put credentials from your instance of **Spark Service** in cell bellow.

In [34]:
spark_credentials={
  "tenant_id": "****",
  "tenant_id_full": "****",
  "cluster_master_url": "https://spark.bluemix.net",
  "tenant_secret": "****",
  "instance_id": "****",
  "plan": "ibm.SparkService.PayGoPersonal"
}

Encode the spark credentials.

In [35]:
spark_instance = {
    "credentials": spark_credentials,
    "version": "2.0"
}
encoded_spark_instance = base64.b64encode(json.dumps(spark_instance))

#### Create streaming deployment for published model

In [36]:
input_topic_name = 'streamingi'
output_topic_name = 'streamingo'

**Action**: Please paste credentials of your **MessageHub Service** to cell bellow.

In [37]:
messagehub_credentials={
  "instance_id": "****",
  "mqlight_lookup_url": "https://mqlight-lookup-prod01.messagehub.services.us-south.bluemix.net/Lookup?serviceId=fa4d499e-b908-4982-84f0-69c30c1b30a0",
  "api_key": "****",
  "kafka_admin_url": "https://kafka-admin-prod01.messagehub.services.us-south.bluemix.net:443",
  "kafka_rest_url": "https://kafka-rest-prod01.messagehub.services.us-south.bluemix.net:443",
  "kafka_brokers_sasl": [
    "kafka03-prod01.messagehub.services.us-south.bluemix.net:9093",
    "kafka04-prod01.messagehub.services.us-south.bluemix.net:9093",
    "kafka01-prod01.messagehub.services.us-south.bluemix.net:9093",
    "kafka02-prod01.messagehub.services.us-south.bluemix.net:9093",
    "kafka05-prod01.messagehub.services.us-south.bluemix.net:9093"
  ],
  "user": "****",
  "password": "****"
}

In [38]:
messagehub_instance = {x: messagehub_credentials[x] for x in messagehub_credentials.keys() if x != "instance_id"}
header_streaming = {'Content-Type': 'application/json', 'Authorization': 'Bearer {token}'.format(token=mltoken), 'X-Spark-Service-Instance': encoded_spark_instance}
payload_streaming = {"type":"stream", "name": "Sentiment Prediction", "description": "Streaming Deployment", 
                     "input": {"connection": messagehub_instance, "source": { "type": "kafka", "topic": input_topic_name}}, 
                     "output": {"connection": messagehub_instance, "target": {"type": "kafka", "topic": output_topic_name}}}

response_streaming = requests.post(endpoint_deployments, json=payload_streaming, headers=header_streaming)

print(response_streaming)
pp(json.loads(response_streaming.text))

<Response [201]>
{ u'entity': { u'deployed_version': { u'created_at': u'2017-09-15T12:09:26.276Z',
                                      u'guid': u'58d2fe5b-7661-459b-b342-a262f5846e61',
                                      u'url': u'https://ibm-watson-ml.mybluemix.net/v2/artifacts/models/c575aac9-1dad-4e29-98d4-1f8c7dc8c764/versions/58d2fe5b-7661-459b-b342-a262f5846e61'},
               u'description': u'Streaming Deployment',
               u'input': { u'connection': {...}, u'source': {...}},
               u'model_type': u'sparkml-model-2.0',
               u'name': u'Sentiment Prediction',
               u'output': { u'connection': {...}, u'target': {...}},
               u'published_model': { u'author': {},
                                     u'created_at': u'2017-09-15T12:12:17.328Z',
                                     u'guid': u'c575aac9-1dad-4e29-98d4-1f8c7dc8c764',
                                     u'name': u'Sentiment Prediction Model',
                                

Now, you have streaming deployment of your model ready to make predictions.

<a id="messagehub"></a>
## 7. Making predictions with streaming deployment

When we want to make predictions with streaming deployment of our model we have to push messages to input topic. Streaming deployment will process that message and write results to output topic.

#### Get the list of topics from MessageHub

In [39]:
messagehub_topic_header = {'X-Auth-Token': messagehub_credentials['api_key']}
messagehub_endpoint = 'https://kafka-rest-prod01.messagehub.services.us-south.bluemix.net:443/topics/'

messagehub_response = requests.get(messagehub_endpoint, headers=messagehub_topic_header)

print(messagehub_response)
pp(json.loads(messagehub_response.text))

<Response [200]>
[u'streamingi', u'streamingo']


**Note**: You should see two topics: **streamingi** and **streamingo** created in setup step.

### Send scoring data to input topic

We have to encode input data to send it to stream.

In [40]:
encoded_payload_value = base64.b64encode(json.dumps({'id': 1, 'text':'My first streaming deployment! It feels so awesome!!'}))

print(encoded_payload_value)

eyJ0ZXh0IjogIk15IGZpcnN0IHN0cmVhbWluZyBkZXBsb3ltZW50ISBJdCBmZWVscyBzbyBhd2Vzb21lISEiLCAiaWQiOiAxfQ==


In order to push encoded data to our topic, we will use below curl call.

In [41]:
messagehub_header = dict({'Content-Type': 'application/vnd.kafka.binary.v1+json'}, **messagehub_topic_header)
input_payload = {'records':[{ 'key': 'null', 'value': encoded_payload_value}]}
messagehub_input_endpoint = messagehub_endpoint + 'streamingi'

response_message_input = requests.post(messagehub_input_endpoint, json=input_payload, headers=messagehub_header)
                                
print(response_message_input)                                
pp(json.loads(response_message_input.text))

<Response [200]>
{ u'key_schema_id': None,
  u'offsets': [ { u'error': None,
                  u'error_code': None,
                  u'offset': 0,
                  u'partition': 0}],
  u'value_schema_id': None}


### Receive scored data from output topic

Create MessageHub consumer

In [42]:
output_consumer_payload = {"format": "binary", "auto.offset.reset": "smallest"}
messagehub_output_consumer_endpoint = 'https://kafka-rest-prod01.messagehub.services.us-south.bluemix.net:443/consumers/my_output_consumer'
messagehub_output_consumer_response = requests.post(messagehub_output_consumer_endpoint, json=output_consumer_payload, headers=messagehub_header)

print(messagehub_output_consumer_response)
pp(json.loads(messagehub_output_consumer_response.text))

<Response [200]>
{ u'base_uri': u'https://kafka-rest-prod01.messagehub.services.us-south.bluemix.net/consumers/my_output_consumer/instances/rest-consumer-kafka-rest.7d5404a6-99e8-11e7-97d9-c25cd54a1102-1d4dbe5a-4d0c-43e3-a3dc-8546a5eb7eae',
  u'instance_id': u'rest-consumer-kafka-rest.7d5404a6-99e8-11e7-97d9-c25cd54a1102-1d4dbe5a-4d0c-43e3-a3dc-8546a5eb7eae'}


Read messages from output topic

In [43]:
messagehub_read_output_endpoint = "{uri}/topics/{output_topic}".format(uri=json.loads(messagehub_output_consumer_response.text).get('base_uri'), output_topic=output_topic_name)
messagehub_read_output_response = requests.get(messagehub_read_output_endpoint, headers=messagehub_header)

print(messagehub_read_output_response)
pp(json.loads(messagehub_read_output_response.text))

<Response [200]>
[ { u'key': None,
    u'offset': 0,
    u'partition': 0,
    u'value': u'eyJpZCI6MSwidGV4dCI6Ik15IGZpcnN0IHN0cmVhbWluZyBkZXBsb3ltZW50ISBJdCBmZWVscyBzbyBhd2Vzb21lISEiLCJ3b3JkcyI6WyJteSIsImZpcnN0Iiwic3RyZWFtaW5nIiwiZGVwbG95bWVudCEiLCJpdCIsImZlZWxzIiwic28iLCJhd2Vzb21lISEiXSwiZmVhdHVyZXMiOnsidHlwZSI6MCwic2l6ZSI6MjYyMTQ0LCJpbmRpY2VzIjpbMzc4NTIsODIxMTEsODYxNzUsMTQxMDYzLDE1MzA3NywxODg0MjQsMTk2NzY2LDI1NjE3N10sInZhbHVlcyI6WzEuMCwxLjAsMS4wLDEuMCwxLjAsMS4wLDEuMCwxLjBdfSwicmF3UHJlZGljdGlvbiI6eyJ0eXBlIjoxLCJ2YWx1ZXMiOlswLjk1MTU4MDk2MDUzNzY0OTcsLTAuOTUxNTgwOTYwNTM3NjQ5N119LCJwcm9iYWJpbGl0eSI6eyJ0eXBlIjoxLCJ2YWx1ZXMiOlswLjcyMTQzMzAxMDc4NTY3MTEsMC4yNzg1NjY5ODkyMTQzMjg5XX0sInByZWRpY3Rpb24iOjAuMH0='}]


Decode the output message

In [44]:
message_output_encoded = json.loads(messagehub_read_output_response.text)[0].get('value')
message_output_decoded = base64.b64decode(message_output_encoded)

pp(message_output_decoded)

'{"id":1,"text":"My first streaming deployment! It feels so awesome!!","words":["my","first","streaming","deployment!","it","feels","so","awesome!!"],"features":{"type":0,"size":262144,"indices":[37852,82111,86175,141063,153077,188424,196766,256177],"values":[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0]},"rawPrediction":{"type":1,"values":[0.9515809605376497,-0.9515809605376497]},"probability":{"type":1,"values":[0.7214330107856711,0.2785669892143289]},"prediction":0.0}'


<a id="summary"></a>
## 8. Summary and next steps     

 You successfully completed this notebook! You learned how to use Apache Spark machine learning as well as Watson Machine Learning for model creation and deployment. Check out our _[Online Documentation](https://console.ng.bluemix.net/docs/services/PredictiveModeling/index.html?pos=2)_ for more samples, tutorials, documentation, how-tos, and blog posts. 

### Authors

**Umit Mert Cakmak**, is a Data Scientist in IBM with a track record of developing enterprise-level applications that substantially increases clients' ability to turn data into actionable knowledge.

Copyright © 2017 IBM. This notebook and its source code are released under the terms of the MIT License.